In [1]:
import pandas as pd, numpy as np
import sys, os
from pprint import pprint
import random
from datetime import datetime, timedelta

In [2]:
import seaborn as sns
from matplotlib import pyplot as plt

In [3]:
def print_runtime():
    print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print("-"*19)
print_runtime()

2021-09-14 23:41:23
-------------------


# 1. GearVN Data

In [41]:
source_path = "./Data-GEARVN.csv"
pdf_gearvn = pd.read_csv(source_path)
print(pdf_gearvn.shape)
display(pdf_gearvn.head())
print_runtime()

(9782, 28)


,ID tích hợp,Thời gian,Kho lấy hàng,Thành phố,Quận huyện,Mã sản phẩm,Ngành,Hãng,Mã chung,Mã riêng,...,Ghi chú của CSKH,Trạng thái,Chăm sóc KH,Thiếu,Thieu,DH thiếu,Ngày xác nhận,Thời gian xác nhận,Nguồn đơn hàng,Column1
0,hd211915,07-02-21 21:48,2 HCM-HHT-KD-V.2,Hồ Chí Minh,Quận Gò Vấp,CHU-LOG-G304,CHU,LOGITECH,NaN,NaN,...,thieu chuot,Thành công,Nguyễn Anh Nguyên,-,-,-,07-02-21 21:55,0:02:04,SHOWROOMHHT,NaN
1,hd211915,07-02-21 21:48,2 HCM-HHT-KD-V.2,Hồ Chí Minh,Quận Gò Vấp,KB-COR-K63-WIRELESS-MX-RD,KB,CORSAIR,NaN,NaN,...,thieu chuot,Thành công,Nguyễn Anh Nguyên,-,-,-,07-02-21 21:55,0:00:00,SHOWROOMHHT,NaN
2,hd211916,07-02-21 21:47,8 HCM-CQ-KD,Hồ Chí Minh,Quận 1,HDD-WD-BLUE-1TB,HDD,WD,NaN,NaN,...,NaN,Thành công,Trần Thiện Tý,-,-,-,07-02-21 21:48,0:00:00,SHOWROOMHHT,NaN
3,hd211914,07-02-21 21:42,2 HCM-HHT-KD-V.2,Bình Dương,Thị xã Tân Uyên,TAI-STE-ARCTIS-7-WHITE,TAI,STEELSERIES,NaN,NaN,...,NaN,Hệ thống hủy,Nguyễn Anh Nguyên,-,-,-,07-02-21 21:44,0:00:00,SHOWROOMHHT,NaN
4,hd211914,07-02-21 21:42,2 HCM-HHT-KD-V.2,Bình Dương,Thị xã Tân Uyên,PK-KING-BINH-GIU-NHIET-BLK,PK,KINGSTON,NaN,NaN,...,NaN,Hệ thống hủy,Nguyễn Anh Nguyên,-,-,-,07-02-21 21:44,0:00:00,SHOWROOMHHT,NaN


2021-09-15 09:22:30
-------------------


In [56]:
pdf_gearvn[["ID", "Modified"]] = pdf_gearvn["ID tích hợp"].str.split("-", expand=True)
# hd211915-3: ID(hd211915); Modified(3)  
# Effect of n: a-b-c-d: n=None/0/-1 => (a, b, c, d)
# n = 1 => (a, b-c-d)

# Modified = None: hd211915 => (hd211915, None)
pdf_gearvn["Modified"] = pdf_gearvn["Modified"].fillna(0).astype(int)
display(pdf_gearvn[["ID tích hợp", "ID", "Modified"]].head())
print_runtime()

,ID tích hợp,ID,Modified
0,hd211915,hd211915,0
1,hd211915,hd211915,0
2,hd211916,hd211916,0
3,hd211914,hd211914,0
4,hd211914,hd211914,0


2021-09-15 10:10:25
-------------------


In [35]:
pdf_gearvn.dtypes

ID tích hợp            object
Thời gian              object
Kho lấy hàng           object
Thành phố              object
Quận huyện             object
Mã sản phẩm            object
Ngành                  object
Hãng                   object
Mã chung               object
Mã riêng               object
Chiết khấu             object
Giá                    object
Giá sau CK             object
Số lượng                int64
Giá trị đơn hàng       object
Tiền chiết khấu        object
Tiền đặt cọc          float64
Tổng thu               object
Ghi chú của CSKH       object
Trạng thái             object
Chăm sóc KH            object
Thiếu                  object
Thieu                  object
DH thiếu               object
Ngày xác nhận          object
Thời gian xác nhận     object
Nguồn đơn hàng         object
Column1               float64
ID                     object
Modified                int64
dtype: object

In [38]:
display(
    pdf_gearvn
    [(pdf_gearvn["Modified"]>1)]
    [["ID tích hợp", "ID", "Modified"]]
    .head()
)

print_runtime()

,ID tích hợp,ID,Modified
121,hd203913-3,hd203913,3
122,hd203913-3,hd203913,3
123,hd203913-3,hd203913,3
124,hd203913-3,hd203913,3
125,hd203913-3,hd203913,3


2021-09-15 00:20:11
-------------------


In [5]:
pdf_agent = pdf_gearvn[["Chăm sóc KH"]].drop_duplicates()
print(f"Number of agents: {pdf_agent.shape[0]}")
display(pdf_agent.head())

print_runtime()

Number of agents: 22


,Chăm sóc KH
0,Nguyễn Anh Nguyên
2,Trần Thiện Tý
5,Phạm Ngọc Hiếu
9,Trần Anh Vũ
12,Võ Quốc Khánh


2021-09-14 23:41:24
-------------------


In [6]:
pdf_gearvn.columns

Index(['ID tích hợp', 'Thời gian', 'Kho lấy hàng', 'Thành phố', 'Quận huyện',
       'Mã sản phẩm', 'Ngành', 'Hãng', 'Mã chung', 'Mã riêng', 'Chiết khấu',
       'Giá', 'Giá sau CK', 'Số lượng', 'Giá trị đơn hàng', 'Tiền chiết khấu',
       'Tiền đặt cọc', 'Tổng thu', 'Ghi chú của CSKH', 'Trạng thái',
       'Chăm sóc KH', 'Thiếu', 'Thieu', 'DH thiếu', 'Ngày xác nhận',
       'Thời gian xác nhận', 'Nguồn đơn hàng', 'Column1'],
      dtype='object')

In [21]:
pdf_gearvn["Trạng thái"].unique()

array(['Thành công', 'Hệ thống hủy', 'Đã xác nhận', 'Đang xác nhận'],
      dtype=object)

# Products

# Question:
- Should we put "Chiết khấu" and "Tiền chiết khấu" into pdf_prod?


In [43]:
# Consider another option
# ls_prod_col = ["Ngành", "Hãng", "Mã sản phẩm", "Giá"]
pdf_prod = pdf_gearvn[[
    "Ngành", "Hãng", "Mã sản phẩm", "Giá"
    # , "Chiết khấu", "Tiền chiết khấu", "Kho lấy hàng"
    ]].drop_duplicates()
print(f"Number of products: {pdf_prod.shape[0]}")
display(pdf_prod.head())

print_runtime()

Number of products: 1166


,Ngành,Hãng,Mã sản phẩm,Giá
0,CHU,LOGITECH,CHU-LOG-G304,"790,000"
1,KB,CORSAIR,KB-COR-K63-WIRELESS-MX-RD,"2,490,000"
2,HDD,WD,HDD-WD-BLUE-1TB,"980,000"
3,TAI,STEELSERIES,TAI-STE-ARCTIS-7-WHITE,"3,990,000"
4,PK,KINGSTON,PK-KING-BINH-GIU-NHIET-BLK,-


2021-09-15 09:34:33
-------------------


- All the Logitech mice G304 have the same price 790000, although they have different "Discount" and "Total discount"
- We can assume that "Total discount" and "Discount" are belong to pdf_orders or pdf_order_details, where "Total discount" is applied for the whole order while "Discount" is "Total discount" divided by number of products.
- If the policy is that, the mouse and kb are calculated in the total price, but then covered by the discount, we can add another column to compare between "Total discount", "Mouse and kb". If they're all matched in the orders which contained both PC parts and gears, then we have a way to compare package included / separated sales for GEAR.

In [40]:
display(pdf_prod[
    (pdf_prod["Ngành"]=="CHU")
    & (pdf_prod["Hãng"]=="LOGITECH")
    & (pdf_prod["Mã sản phẩm"]=="CHU-LOG-G304")
])

,Ngành,Hãng,Mã sản phẩm,Kho lấy hàng,Chiết khấu,Giá,Tiền chiết khấu
0,CHU,LOGITECH,CHU-LOG-G304,2 HCM-HHT-KD-V.2,0,790000,0
634,CHU,LOGITECH,CHU-LOG-G304,8 HCM-CQ-KD,36443,790000,1770000
920,CHU,LOGITECH,CHU-LOG-G304,2 HCM-HHT-KD-V.2,17133,790000,550000
1992,CHU,LOGITECH,CHU-LOG-G304,2 HCM-HHT-KD-V.2,123582,790000,4000000
2748,CHU,LOGITECH,CHU-LOG-G304,6 HN-TH-KD,0,790000,0
2751,CHU,LOGITECH,CHU-LOG-G304,8 HCM-CQ-KD,0,790000,0
4319,CHU,LOGITECH,CHU-LOG-G304,6 HN-TH-KD,15521,790000,300000
4461,CHU,LOGITECH,CHU-LOG-G304,3 HCM-HVT-KD,0,790000,0
8248,CHU,LOGITECH,CHU-LOG-G304,2 HCM-HHT-KD-V.2,0,790000,1040000


In [19]:
pdf_prod[
    (pdf_prod["Ngành"]=="KB")
    & (pdf_prod["Hãng"]=="CORSAIR")
    & (pdf_prod["Mã sản phẩm"]=="KB-COR-K63-WIRELESS-MX-RD")
]

,Ngành,Hãng,Mã sản phẩm,Kho lấy hàng,Chiết khấu,Giá,Tiền chiết khấu
1,KB,CORSAIR,KB-COR-K63-WIRELESS-MX-RD,2 HCM-HHT-KD-V.2,0,2490000,0
5572,KB,CORSAIR,KB-COR-K63-WIRELESS-MX-RD,3 HCM-HVT-KD,0,2490000,0


In [ ]:
# Convert Giá, Chiết khấu, Tiền chiết khấu to int

In [ ]:
pdf_gearvn["Chiết khấu"] = (
    pdf_gearvn["Chiết khấu"].str.replace(",", "")
    .fillna(0.0)
    .astype(int)
)

pdf_gearvn["Tiền chiết khấu"] = (
    pdf_gearvn["Tiền chiết khấu"].str.replace(",", "")
    .fillna(0.0)
    .astype(int)
)
pdf_gearvn["Giá"] = (
    pdf_gearvn["Giá"].str.replace(",", "").replace(" -   ", "0")
    .fillna(0.0)
    .astype(int)
)


In [60]:

pdf_gearvn["Giá trị đơn hàng"] = (
    pdf_gearvn["Giá trị đơn hàng"].str.replace(",", "").replace(" -   ", "0")
    .fillna(0.0)
    .astype(int)
)

pdf_gearvn["Tổng thu"] = (
    pdf_gearvn["Tổng thu"].str.replace(",", "").replace(" -   ", "0")
    .fillna(0.0)
    .astype(int)
)


In [52]:
pdf_gearvn[
    (pdf_gearvn["Trạng thái"].str.contains("Hệ thống")) # Return list of indexes where `Trang thai` contains "He thong huy"
    & (pdf_gearvn["Ngành"]=="TAI")  # List of indexes where `Nganh` == `TAI`
    # & => Combine 2 (or more) list of indexes
].head()

,ID tích hợp,Thời gian,Kho lấy hàng,Thành phố,Quận huyện,Mã sản phẩm,Ngành,Hãng,Mã chung,Mã riêng,...,Ghi chú của CSKH,Trạng thái,Chăm sóc KH,Thiếu,Thieu,DH thiếu,Ngày xác nhận,Thời gian xác nhận,Nguồn đơn hàng,Column1
3,hd211914,07-02-21 21:42,2 HCM-HHT-KD-V.2,Bình Dương,Thị xã Tân Uyên,TAI-STE-ARCTIS-7-WHITE,TAI,STEELSERIES,NaN,NaN,...,NaN,Hệ thống hủy,Nguyễn Anh Nguyên,-,-,-,07-02-21 21:44,0:00:00,SHOWROOMHHT,NaN
91,hd211894,07-02-21 20:53,2 HCM-HHT-KD-V.2,Hồ Chí Minh,huyện Củ Chi,TAI-DAR-EH416-RGB,TAI,DAREU,NaN,NaN,...,NaN,Hệ thống hủy,NaN,-,-,-,NaN,0:10:10,TELEPHONE,NaN
154,hd211877,07-02-21 19:57,3 HCM-HVT-KD,Hồ Chí Minh,huyện Củ Chi,TAI-DAR-EH925-PRO-RGB,TAI,DAREU,NaN,NaN,...,NaN,Hệ thống hủy,Trần Anh Vũ,-,-,-,07-02-21 19:59,0:01:53,SHOWROOMHHT,NaN
155,hd211875,07-02-21 19:56,2 HCM-HHT-KD-V.2,Hồ Chí Minh,Quận Gò Vấp,TAI-PAM-PADMATE-SLIDE-PLUS-GRN,TAI,PAMU,NaN,NaN,...,NaN,Hệ thống hủy,Nguyễn Cao Thắng,-,-,-,07-02-21 19:58,0:01:53,OTHER,NaN
320,hd211814,07-02-21 18:12,3 HCM-HVT-KD,NaN,NaN,TAI-RAZ-BLACK-SHARK-V2-X,TAI,RAZER,NaN,NaN,...,NaN,Hệ thống hủy,Trần Anh Vũ,-,-,-,08-02-21 8:18,0:18:15,HARAVAN,NaN


In [47]:
display(pdf_gearvn[["Giá","Chiết khấu","Tiền chiết khấu"]].head(10))

,Giá,Chiết khấu,Tiền chiết khấu
0,790000,0,0
1,2490000,0,0
2,980000,0,0
3,3990000,350000,350000
4,0,0,350000
5,1490000,0,0
6,1520000,0,0
7,1490000,0,0
8,2790000,0,0
9,435000,0,0


In [48]:
pdf_gearvn.dtypes

ID tích hợp            object
Thời gian              object
Kho lấy hàng           object
Thành phố              object
Quận huyện             object
Mã sản phẩm            object
Ngành                  object
Hãng                   object
Mã chung               object
Mã riêng               object
Chiết khấu              int64
Giá                     int64
Giá sau CK             object
Số lượng                int64
Giá trị đơn hàng       object
Tiền chiết khấu         int64
Tiền đặt cọc          float64
Tổng thu               object
Ghi chú của CSKH       object
Trạng thái             object
Chăm sóc KH            object
Thiếu                  object
Thieu                  object
DH thiếu               object
Ngày xác nhận          object
Thời gian xác nhận     object
Nguồn đơn hàng         object
Column1               float64
dtype: object

# Order and order detail

In [57]:
pdf_gearvn.columns

Index(['ID tích hợp', 'Thời gian', 'Kho lấy hàng', 'Thành phố', 'Quận huyện',
       'Mã sản phẩm', 'Ngành', 'Hãng', 'Mã chung', 'Mã riêng', 'Chiết khấu',
       'Giá', 'Giá sau CK', 'Số lượng', 'Giá trị đơn hàng', 'Tiền chiết khấu',
       'Tiền đặt cọc', 'Tổng thu', 'Ghi chú của CSKH', 'Trạng thái',
       'Chăm sóc KH', 'Thiếu', 'Thieu', 'DH thiếu', 'Ngày xác nhận',
       'Thời gian xác nhận', 'Nguồn đơn hàng', 'Column1', 'ID', 'Modified'],
      dtype='object')

In [61]:
ls_num_col = {
    "Giá trị đơn hàng": 0.0, "Tiền chiết khấu": 0.0, 
    "Tiền đặt cọc": 0.0, "Tổng thu": 0.
}
ls_str_col = {
    "Thời gian": "", "Ghi chú của CSKH": "", "Trạng thái": "", "Chăm sóc KH": "",
    "Thiếu": "", "Thieu": "", "DH thiếu": "",
    "Ngày xác nhận": "","Nguồn đơn hàng": ""
}
pdf_order = (
    pdf_gearvn[[
        "ID tích hợp", "ID", "Modified",
        "Thời gian", # Sales created order
        "Giá trị đơn hàng", "Tiền chiết khấu", "Tiền đặt cọc", "Tổng thu",
        "Ghi chú của CSKH", "Trạng thái", "Chăm sóc KH", "Thiếu", "Thieu", "DH thiếu", 
        "Ngày xác nhận", # Storage confirmed timestamp
        "Nguồn đơn hàng",]]
    .fillna(ls_num_col)
    .fillna(ls_str_col)
    .groupby(["ID tích hợp", "ID", "Modified"], as_index=False).agg({
        "Thời gian": "max", 
        "Giá trị đơn hàng": "max", 
        "Tiền chiết khấu": "max", 
        "Tiền đặt cọc": "max", 
        "Tổng thu": "max",
        "Ghi chú của CSKH": "max", 
        "Trạng thái": "max", 
        "Chăm sóc KH": "max", 
        "Thiếu": "max", 
        "Thieu": "max", 
        "DH thiếu": "max", 
        "Ngày xác nhận": "max", 
        "Nguồn đơn hàng": "max"
    })
)

print(f"Number of order: {pdf_order.shape[0]}")
display(pdf_order.head())

print_runtime()

Number of order: 3400


,ID tích hợp,ID,Modified,Thời gian,Giá trị đơn hàng,Tiền chiết khấu,Tiền đặt cọc,Tổng thu,Ghi chú của CSKH,Trạng thái,Chăm sóc KH,Thiếu,Thieu,DH thiếu,Ngày xác nhận,Nguồn đơn hàng
0,hd108850-9,hd108850,9,05-02-21 19:46,294000000,0,0.0,294000000,,Hệ thống hủy,Nguyễn Anh Tuấn,1,-,1,06-02-21 7:58,OTHER
1,hd170656-6,hd170656,6,06-02-21 18:11,193500000,1270000,0.0,193500000,,Hệ thống hủy,Nguyễn Thế Vinh,-,-,-,06-02-21 18:17,OTHER
2,hd203577-1,hd203577,1,01-02-21 8:11,17830000,0,0.0,17830000,,Thành công,Nguyễn Thế Nam,-,-,-,02-02-21 12:52,SHOWROOMHHT
3,hd203879-1,hd203879,1,01-02-21 8:43,0,9000000,0.0,0,,Thành công,Lê Quang Hoàng Dũng,-,-,-,02-04-21 9:05,SHOWROOMHHT
4,hd203913-2,hd203913,2,05-02-21 18:41,147250003,4329997,0.0,147250003,,Hệ thống hủy,,-,-,-,,OTHER


2021-09-15 10:15:14
-------------------


### Need to clear the relationship between "Chiết khấu", "Giá", "Giá sau CK" as some records showed inconsistent value

In [62]:
pdf_gearvn.columns

Index(['ID tích hợp', 'Thời gian', 'Kho lấy hàng', 'Thành phố', 'Quận huyện',
       'Mã sản phẩm', 'Ngành', 'Hãng', 'Mã chung', 'Mã riêng', 'Chiết khấu',
       'Giá', 'Giá sau CK', 'Số lượng', 'Giá trị đơn hàng', 'Tiền chiết khấu',
       'Tiền đặt cọc', 'Tổng thu', 'Ghi chú của CSKH', 'Trạng thái',
       'Chăm sóc KH', 'Thiếu', 'Thieu', 'DH thiếu', 'Ngày xác nhận',
       'Thời gian xác nhận', 'Nguồn đơn hàng', 'Column1', 'ID', 'Modified'],
      dtype='object')

In [64]:
ls_od_col = [
    "ID tích hợp", "Mã sản phẩm", "Số lượng", "Chiết khấu", "Giá",
    "ID", "Modified"
]

pdf_orderdetail = pdf_gearvn[pdf_gearvn["Trạng thái"]=="Thành công"][ls_od_col]
display(pdf_orderdetail.head())
print_runtime()

,ID tích hợp,Mã sản phẩm,Số lượng,Chiết khấu,Giá,ID,Modified
0,hd211915,CHU-LOG-G304,1,0,790000,hd211915,0
1,hd211915,KB-COR-K63-WIRELESS-MX-RD,1,0,2490000,hd211915,0
2,hd211916,HDD-WD-BLUE-1TB,1,0,980000,hd211916,0
5,hd211912,CHU-ZOW-BENQ-S1,1,0,1490000,hd211912,0
6,hd211911,KB-AKKO-DRAGONBALL-ACG84-AKKO-PINK,1,0,1520000,hd211911,0


2021-09-15 10:20:36
-------------------


In [65]:
pdf_orderdetail.shape

(7612, 7)

# Group by ID: Only orders with "Trạng thái" == "Thành công"
# Separated data into
pdf_prod: Finalized => Save to csv/pickle file
  pdf_prod.to_pickle(filename)
pdf_order
pdf_orderdetail

## HW0: Create new column "Giá sau CK" = "Giá" - "Chiết khấu"

# Question 1. 
- Tính tỉ trọng đơn hàng Online và Offline trên doanh thu tổng của ngành hàng
- Theo GEARVN quy định, Sales của bộ phận Showroom lên đơn sẽ tính là đơn hàng Offline, áp dụng tương tự với Online.
pdf_agent["Chăm sóc KH", "IS_ONLINE"]
pdf_gearvn.join(pdf_agent, on="Chăm sóc KH", how="inner")
=> pdf_prod, pdf_order, pdf_orderdetail 
## HW1: Only get Sucessful order
pdf_orderdetail.groupby("IS_ONLINE").agg(
    "Giá sau CK": "sum"
)

# Question 2. Xác định tỉ trọng sản phẩm Gaming Gear bán lẻ và sản phẩm Gaming Gear được bán cùng PC/Monitor
- Gaming Gear bán lẻ: Chiết khấu = 0
- Bán kèm: Chiết khấu !=0
pdf_gearvn["with_pc"] = (pdf_gearvn["Chiết khấu"] != 0).astype(int)

- Separate Gaming Gear only
  - Method 1:
pdf_gaming_gear = pdf_gearvn[
    (pdf_gearvn["Ngành]=="TAI")
    | (pdf_gearvn["Ngành]=="CHU")
    | (pdf_gearvn["Ngành]=="KB")]
]
  - Method 2:
pdf_gaming_gear = pdf_gearvn[
    pdf_gearvn["Ngành].isin(["TAI", "CHU", "KB"])
]
## HW2: Compare "Giá" vs. "Giá sau CK"
pdf_gg_compare = (
    pdf_gaming_gear
    .groupby("with_pc", as_index=False)
    .agg({"Giá sau CK": "sum"})
)

